# Ta mère la pute

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [11]:
import datetime
from helpers import *

train_path='train.csv'
train_target,train_set=load_data(train_path)

test_path='train.csv'
test_target,test_set=load_data(test_path)


## Modeling

In [13]:
train_target,train_set =build_model_data(train_target,train_set)


array([[   1.   ,  138.47 ,   51.655, ...,    1.24 ,   -2.475,  113.497],
       [   1.   ,  160.937,   68.768, ..., -999.   , -999.   ,   46.226],
       [   1.   , -999.   ,  162.172, ..., -999.   , -999.   ,   44.251],
       ...,
       [   1.   ,  105.457,   60.526, ..., -999.   , -999.   ,   41.992],
       [   1.   ,   94.951,   19.362, ..., -999.   , -999.   ,    0.   ],
       [   1.   , -999.   ,   72.756, ..., -999.   , -999.   ,    0.   ]])

In [25]:
#Compute the loss using mse
def compute_loss(y, tx, w):
    e=y-tx.dot(w)
    n=len(y)
    return 1/(2*n)*e.dot(e)

#Compute the gradient of the loss function (mse)
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e=y-tx.dot(w)
    grd=tx.T.dot(e)
    return -(1/len(y))*grd 

In [40]:
#Gradient descent algorithm
def least_squares_GD(y, tx, initial_w,max_iters, gamma):

    w=initial_w
    
    for n_iter in range(max_iters):
        grd=compute_gradient(y,tx,w)
        w=w-gamma*grd
        
        loss=compute_loss(y,tx,w)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    
    loss=compute_loss(y,tx,w)
    return w,loss
    

In [27]:
def compute_stoch_least_squares_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""   
    my=None
    mx=None
    for mmini_y,mini_x in batch_iter(y, tx, batch_size=1):
            my=mmini_y
            mx=mini_x
    
    return compute_gradient(my, mx, w)


def least_squares_SGD(y, tx, initial_w, batch_size, max_iters, gamma):
    "Gradient descent algorithm"
    w=initial_w
    for i in range(max_iters):
        print(i)
        grd=gamma*compute_stoch_least_squares_gradient(y,tx,w)
        w=w-grd
    loss=compute_loss(y,tx,w)
    
    return w, loss

### First try remonving all missing values

In [46]:
#indices of all data points that does'nt contain any values equal to -999 (unknown values)
indices=np.all(train_set>-999,axis=1)

In [57]:
max_iters = 1000
gamma = 0.000002
w_initial = np.zeros(len(train_set[0]))

w,loss=least_squares_GD(train_target[indices],train_set[indices], w_initial,max_iters, gamma)

Gradient Descent(0/999): loss=0.1255883562336643, w0=9.364888275538068e-07, w1=0.00011610844434330688
Gradient Descent(1/999): loss=0.12458805928630982, w0=9.88824537888116e-07, w1=0.00012220319537859523
Gradient Descent(2/999): loss=0.12425665067943972, w0=1.110573760714261e-06, w1=0.00013679844351869924
Gradient Descent(3/999): loss=0.12394834561885922, w0=1.227602891569275e-06, w1=0.00015067711754057135
Gradient Descent(4/999): loss=0.12365507323299686, w0=1.345349328803308e-06, w1=0.0001645131020356297
Gradient Descent(5/999): loss=0.12337398447437634, w0=1.463156801766168e-06, w1=0.00017822377089948322
Gradient Descent(6/999): loss=0.12310317323872233, w0=1.5808896287874963e-06, w1=0.00019179189245959247
Gradient Descent(7/999): loss=0.12284133106350233, w0=1.6984113834867414e-06, w1=0.00020520068886154273
Gradient Descent(8/999): loss=0.12258753832814906, w0=1.8156174175618217e-06, w1=0.0002184377625783324
Gradient Descent(9/999): loss=0.12234112928932457, w0=1.9324260725092066e-

Gradient Descent(92/999): loss=0.11327883542756667, w0=1.0005456614253423e-05, w1=0.0008097007467375797
Gradient Descent(93/999): loss=0.11323267885484814, w0=1.0090046908290887e-05, w1=0.0008128584079678788
Gradient Descent(94/999): loss=0.11318716983383466, w0=1.0174465100897324e-05, w1=0.0008159646249204673
Gradient Descent(95/999): loss=0.11314229458913581, w0=1.0258714161795542e-05, w1=0.0008190202957933029
Gradient Descent(96/999): loss=0.11309803971891615, w0=1.0342797006453362e-05, w1=0.0008220263026440989
Gradient Descent(97/999): loss=0.11305439218310491, w0=1.0426716497087418e-05, w1=0.000824983511685448
Gradient Descent(98/999): loss=0.11301133929200992, w0=1.051047544364832e-05, w1=0.0008278927735745003
Gradient Descent(99/999): loss=0.11296886869532163, w0=1.0594076604787506e-05, w1=0.0008307549236972964
Gradient Descent(100/999): loss=0.11292696837149305, w0=1.0677522688806159e-05, w1=0.0008335707824478533
Gradient Descent(101/999): loss=0.11288562661748229, w0=1.0760816

Gradient Descent(181/999): loss=0.11072819491077948, w0=1.70976571299391e-05, w1=0.0009615386956879747
Gradient Descent(182/999): loss=0.11071035889065399, w0=1.717423968752778e-05, w1=0.0009623445724555957
Gradient Descent(183/999): loss=0.11069266606944358, w0=1.7250781792905952e-05, w1=0.000963139258531034
Gradient Descent(184/999): loss=0.11067511452995615, w0=1.7327284036248947e-05, w1=0.0009639229380002989
Gradient Descent(185/999): loss=0.1106577023884152, w0=1.7403746997350246e-05, w1=0.0009646957917733948
Gradient Descent(186/999): loss=0.11064042779373962, w0=1.748017124581117e-05, w1=0.000965457997641032
Gradient Descent(187/999): loss=0.11062328892684221, w0=1.7556557341227026e-05, w1=0.0009662097303303039
Gradient Descent(188/999): loss=0.1106062839999467, w0=1.7632905833369824e-05, w1=0.0009669511615593479
Gradient Descent(189/999): loss=0.11058941125592255, w0=1.7709217262367605e-05, w1=0.0009676824600910112
Gradient Descent(190/999): loss=0.11057266896763715, w0=1.77854

Gradient Descent(270/999): loss=0.10954357239984151, w0=2.3802977137998963e-05, w1=0.0010044529310371292
Gradient Descent(271/999): loss=0.10953355649132446, w0=2.3877439879535207e-05, w1=0.0010047276198761382
Gradient Descent(272/999): loss=0.10952359227510362, w0=2.3951889140678945e-05, w1=0.0010049996036674378
Gradient Descent(273/999): loss=0.10951367925462521, w0=2.4026325051322223e-05, w1=0.0010052689234659757
Gradient Descent(274/999): loss=0.10950381693992037, w0=2.4100747739292484e-05, w1=0.0010055356196653107
Gradient Descent(275/999): loss=0.10949400484750253, w0=2.4175157330388848e-05, w1=0.0010057997320088547
Gradient Descent(276/999): loss=0.10948424250026653, w0=2.4249553948417723e-05, w1=0.001006061299600916
Gradient Descent(277/999): loss=0.1094745294273891, w0=2.432393771522776e-05, w1=0.0010063203609175459
Gradient Descent(278/999): loss=0.1094648651642317, w0=2.4398308750744185e-05, w1=0.0010065769538171935
Gradient Descent(279/999): loss=0.10945524925224458, w0=2.4

Gradient Descent(358/999): loss=0.10881768712484285, w0=3.0314662023105818e-05, w1=0.001021585267414642
Gradient Descent(359/999): loss=0.10881085790857659, w0=3.038827320829807e-05, w1=0.0010217265066595907
Gradient Descent(360/999): loss=0.1088040537714147, w0=3.046187725474206e-05, w1=0.0010218670031142665
Gradient Descent(361/999): loss=0.1087972745410977, w0=3.053547419831516e-05, w1=0.0010220067671756429
Gradient Descent(362/999): loss=0.1087905200472001, w0=3.060906407443067e-05, w1=0.0010221458090858737
Gradient Descent(363/999): loss=0.10878379012110594, w0=3.068264691804511e-05, w1=0.0010222841389347156
Gradient Descent(364/999): loss=0.10877708459598442, w0=3.075622276366548e-05, w1=0.00102242176666191
Gradient Descent(365/999): loss=0.10877040330676636, w0=3.082979164535631e-05, w1=0.0010225587020595255
Gradient Descent(366/999): loss=0.10876374609012035, w0=3.090335359674662e-05, w1=0.0010226949547742603
Gradient Descent(367/999): loss=0.10875711278442995, w0=3.09769086510

...

Gradient Descent(534/999): loss=0.10789625255527958, w0=4.318082187670938e-05, w1=0.0010404646387176136
Gradient Descent(535/999): loss=0.10789220513234499, w0=4.325348747355341e-05, w1=0.0010405552264768762
Gradient Descent(536/999): loss=0.1078881677562839, w0=4.332614868154089e-05, w1=0.0010406457290074768
Gradient Descent(537/999): loss=0.1078841403867967, w0=4.3398805506624205e-05, w1=0.0010407361470644051
Gradient Descent(538/999): loss=0.10788012298383019, w0=4.34714579547069e-05, w1=0.0010408264813904274
Gradient Descent(539/999): loss=0.10787611550757553, w0=4.354410603164417e-05, w1=0.001040916732716251
Gradient Descent(540/999): loss=0.10787211791846593, w0=4.361674974324328e-05, w1=0.0010410069017606873
Gradient Descent(541/999): loss=0.10786813017717443, w0=4.368938909526409e-05, w1=0.0010410969892308132
Gradient Descent(542/999): loss=0.1078641522446118, w0=4.3762024093419404e-05, w1=0.0010411869958221285
Gradient Descent(543/999): loss=0.10786018408192445, w0=4.383465474

Gradient Descent(630/999): loss=0.10754845875529308, w0=5.0137418349581246e-05, w1=0.001048848050221809
Gradient Descent(631/999): loss=0.10754522119748451, w0=5.02096840491042e-05, w1=0.0010489325900258292
Gradient Descent(632/999): loss=0.10754199069244368, w0=5.028194576043331e-05, w1=0.0010490170791512404
Gradient Descent(633/999): loss=0.10753876721618925, w0=5.0354203486363284e-05, w1=0.001049101517698214
Gradient Descent(634/999): loss=0.10753555074485473, w0=5.0426457229667534e-05, w1=0.0010491859057637678
Gradient Descent(635/999): loss=0.10753234125468752, w0=5.049870699309835e-05, w1=0.0010492702434418148
Gradient Descent(636/999): loss=0.10752913872204833, w0=5.057095277938707e-05, w1=0.0010493545308232091
Gradient Descent(637/999): loss=0.10752594312341021, w0=5.064319459124424e-05, w1=0.0010494387679957935
Gradient Descent(638/999): loss=0.10752275443535785, w0=5.071543243135979e-05, w1=0.0010495229550444454
Gradient Descent(639/999): loss=0.10751957263458692, w0=5.078766

Gradient Descent(725/999): loss=0.10726938739127105, w0=5.698518028086016e-05, w1=0.0010566592066396032
Gradient Descent(726/999): loss=0.10726672651692516, w0=5.705707710068657e-05, w1=0.001056739066251391
Gradient Descent(727/999): loss=0.10726407084017923, w0=5.712897012235979e-05, w1=0.0010568188757134724
Gradient Descent(728/999): loss=0.10726142034525213, w0=5.720085934724848e-05, w1=0.0010568986349769013
Gradient Descent(729/999): loss=0.10725877501642693, w0=5.727274477671133e-05, w1=0.001056978343992252
Gradient Descent(730/999): loss=0.10725613483805052, w0=5.734462641209712e-05, w1=0.0010570580027096349
Gradient Descent(731/999): loss=0.10725349979453325, w0=5.74165042547448e-05, w1=0.0010571376110787083
Gradient Descent(732/999): loss=0.10725086987034871, w0=5.748837830598358e-05, w1=0.0010572171690486948
Gradient Descent(733/999): loss=0.10724824505003325, w0=5.756024856713301e-05, w1=0.0010572966765683936
Gradient Descent(734/999): loss=0.10724562531818571, w0=5.763211503

Gradient Descent(823/999): loss=0.10703113628724711, w0=6.401319308364556e-05, w1=0.001064238712712133
Gradient Descent(824/999): loss=0.10702891842782196, w0=6.408472293060602e-05, w1=0.0010643133930551267
Gradient Descent(825/999): loss=0.10702670447948215, w0=6.415624907667908e-05, w1=0.0010643880177959144
Gradient Descent(826/999): loss=0.10702449443140155, w0=6.422777152256935e-05, w1=0.0010644625868843497
Gradient Descent(827/999): loss=0.10702228827279348, w0=6.429929026897729e-05, w1=0.0010645371002705687
Gradient Descent(828/999): loss=0.1070200859929104, w0=6.437080531659925e-05, w1=0.0010646115579049924
Gradient Descent(829/999): loss=0.10701788758104394, w0=6.444231666612754e-05, w1=0.0010646859597383304
Gradient Descent(830/999): loss=0.10701569302652439, w0=6.451382431825044e-05, w1=0.0010647603057215831
Gradient Descent(831/999): loss=0.10701350231872093, w0=6.458532827365227e-05, w1=0.0010648345958060455
Gradient Descent(832/999): loss=0.10701131544704107, w0=6.46568285

Gradient Descent(920/999): loss=0.10683274355716198, w0=7.093444889584714e-05, w1=0.0010712175555646606
Gradient Descent(921/999): loss=0.10683085991118088, w0=7.100562255590028e-05, w1=0.001071286658135112
Gradient Descent(922/999): loss=0.10682897928620147, w0=7.107679256946046e-05, w1=0.0010713557018420103
Gradient Descent(923/999): loss=0.10682710167451198, w0=7.114795893699046e-05, w1=0.001071424686670197
Gradient Descent(924/999): loss=0.10682522706842648, w0=7.121912165895197e-05, w1=0.0010714936126048989
Gradient Descent(925/999): loss=0.10682335546028476, w0=7.129028073580551e-05, w1=0.001071562479631727
Gradient Descent(926/999): loss=0.10682148684245245, w0=7.136143616801053e-05, w1=0.0010716312877366763
Gradient Descent(927/999): loss=0.1068196212073205, w0=7.143258795602541e-05, w1=0.0010717000369061256
Gradient Descent(928/999): loss=0.10681775854730548, w0=7.150373610030744e-05, w1=0.001071768727126837
Gradient Descent(929/999): loss=0.10681589885484925, w0=7.15748806013